## facematch

Aqui, espero conseguir mesmo que de maneira rudimentar encontrar os vetores que represetam os dois rostos. Para isso, pretendo utilizar o 'pacote' https://github.com/DefTruth/torchlm.

In [9]:
import numpy as np
import pandas as pd

In [8]:
#load parquet

df = pd.read_parquet('../base/filtered_data.parquet')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5173 entries, 0 to 23593
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  5173 non-null   object
 1   approved            5173 non-null   bool  
 2   analysis_status     5154 non-null   object
 3   kind                5173 non-null   object
 4   month               5173 non-null   int32 
 5   registration_image  5173 non-null   object
 6   comparision_image   5173 non-null   object
dtypes: bool(1), int32(1), object(5)
memory usage: 267.7+ KB
